In [1]:
import json
import numpy as np

Labels = []
with open('./Answers/QA.jsonl', 'r', encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        Labels.append(data['A'])
Answers = Labels[:]

# 1. Rouge

In [13]:
from rouge import Rouge 

def Rouge_Score(hypothesis, reference):
    rouger = Rouge()
    scores = rouger.get_scores(hypothesis, reference)
    arr_1 = np.array([[item['rouge-1']['r'], item['rouge-1']['p'], item['rouge-1']['f']] for item in scores])
    avg_r_1, avg_p_1, avg_f_1 = np.mean(arr_1[:, 0]), np.mean(arr_1[:, 1]), np.mean(arr_1[:, 2])
    arr_2 = np.array([[item['rouge-2']['r'], item['rouge-2']['p'], item['rouge-2']['f']] for item in scores])
    avg_r_2, avg_p_2, avg_f_2 = np.mean(arr_2[:, 0]), np.mean(arr_2[:, 1]), np.mean(arr_2[:, 2])
    arr_l = np.array([[item['rouge-l']['r'], item['rouge-l']['p'], item['rouge-l']['f']] for item in scores])
    avg_r_l, avg_p_l, avg_f_l = np.mean(arr_l[:, 0]), np.mean(arr_l[:, 1]), np.mean(arr_l[:, 2])
    avg_scores = {
        'rouge-1': {'r': avg_r_1, 'p': avg_p_1, 'f': avg_f_1},
        'rouge-2': {'r': avg_r_2, 'p': avg_p_2, 'f': avg_f_2}, 
        'rouge-l': {'r': avg_r_l, 'p': avg_p_l, 'f': avg_f_l}
        }
    return avg_scores, arr_l[:, 2]
rouge_scores, rouge_scores_l_f1 = Rouge_Score(Answers, Labels)
rouge_scores_l_f1

{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.9999999949999997},
 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.9999999949999997},
 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.9999999949999997}}

# 2. BLEU

In [3]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu


def Bleu_Scores(candidates, references):
    references  = np.array(references).reshape(-1, 1)
    references_tokens = [[s.split() for s in sentences] for sentences in references]

    candidates_tokens = [sentence.split() for sentence in candidates]

    bleu_scores = [corpus_bleu(references_tokens, candidates_tokens, weights=(n,)) for n in range(1, 5)]
    bleu_1_scores = [sentence_bleu(r, z, weights=(1,)) for r, z in zip(references_tokens, candidates_tokens)]
    ave_scores = {}
    for n, score in enumerate(bleu_scores, start=1):
       ave_scores[f"BLEU-{n}"] = score
    return ave_scores, bleu_1_scores

Bleu_Scores(Answers, Labels)

({'BLEU-1': 1.0, 'BLEU-2': 1.0, 'BLEU-3': 1.0, 'BLEU-4': 1.0},
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,

# 3. NIST

In [16]:
from nltk.translate.nist_score import corpus_nist, sentence_nist

def Nist_Scores(candidates, references):
    references  = np.array(references).reshape(-1, 1)
    references_tokens = [[s.split() for s in sentences] for sentences in references]
    
    candidates_tokens = [sentence.split() for sentence in candidates]

    nist_5 = corpus_nist(references_tokens, candidates_tokens, n=5)

    nist_5_scores = [sentence_nist(r, z, n=min(len(r), 5)) for r, z in zip(references_tokens, candidates_tokens)]

    return nist_5 , nist_5_scores

Nist_Scores(Answers, Labels)

({'NIST-5': 11.610220208186881},
 [4.221928094887364,
  3.807354922057605,
  4.247927513443585,
  3.664497779200462,
  3.625,
  3.807354922057605,
  4.696131794257842,
  4.221928094887364,
  4.3685225277282065,
  4.16992500144231,
  4.058813890331199,
  2.584962500721156,
  4.373660689688185,
  3.1699250014423126,
  4.436605434317882,
  4.0588138903312,
  4.623516641218017,
  4.0,
  4.623516641218017,
  3.875,
  4.418295834054492,
  4.800812842575149,
  5.045301086491799,
  5.087462841250338,
  5.131556065016097,
  5.11364956559369,
  4.696131794257842,
  5.271928094887366,
  2.584962500721156,
  2.0,
  3.3219280948873626,
  4.142664355548848,
  3.3219280948873626,
  3.807354922057605,
  3.0,
  3.0,
  4.16992500144231,
  3.664497779200462,
  3.9698157824268114,
  3.1699250014423126,
  2.584962500721156,
  3.0,
  4.2626923908396215,
  3.7004397181410913,
  3.947702779220088,
  3.807354922057605,
  3.7946534735443405,
  4.8125,
  2.584962500721156,
  4.825164052322358,
  4.67569813536798

# 4. Meteor

In [18]:
from nltk.translate.meteor_score import meteor_score

def Meteor_Scores(candidates, references):
    references  = np.array(references).reshape(-1, 1)
    references_tokens = [[s.split() for s in sentences] for sentences in references]

    candidates_tokens = [sentence.split() for sentence in candidates]

    meteor_scores = [meteor_score(ref, candidate) for ref, candidate in zip(references_tokens, candidates_tokens)]
    return np.mean(meteor_scores).item(), meteor_scores

Meteor_Scores(Answers, Labels)

({'METEOR': 0.999705915324945},
 [0.9999375,
  0.9998177842565598,
  0.9999271030762502,
  0.9998177842565598,
  0.9998779296875,
  0.9998177842565598,
  0.9999832164076399,
  0.9999375,
  0.9999530428249437,
  0.9999142661179699,
  0.9999142661179699,
  0.9976851851851852,
  0.999968,
  0.9993141289437586,
  0.999958905235473,
  0.9999142661179699,
  0.9999715521165226,
  0.9998779296875,
  0.9999715521165226,
  0.9998779296875,
  0.9999638310185185,
  0.9999832164076399,
  0.9999948631544341,
  0.9999872786484836,
  0.9999915709974881,
  0.9999927453171021,
  0.9999832164076399,
  0.9999921875,
  0.9976851851851852,
  0.9921875,
  0.9995,
  0.9999271030762502,
  0.9995,
  0.9998177842565598,
  0.9990234375,
  0.9990234375,
  0.9999142661179699,
  0.9998177842565598,
  0.999898229187869,
  0.9993141289437586,
  0.9976851851851852,
  0.9990234375,
  0.999958905235473,
  0.9997724169321802,
  0.9999142661179699,
  0.9998177842565598,
  0.9999142661179699,
  0.9999847412109375,
  0.99768

# 5. Bert Score

In [3]:
from bert_score import score


def Bert_Score(candidates, references):

    P, R, F1 = score(candidates, references, model_type="roberta-large", verbose=False)

    return {"BERT-P:": float(np.mean(P.numpy())), "BERT-R:": float(np.mean(R.numpy())), "BERT-F1:": float(np.mean(F1.numpy()))}, F1.numpy()

Bert_Score(Answers, Labels)


Some weights of the model checkpoint at D:\yuyouyu\CODE\QA_Evaluate\models--roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 0.29 seconds, 502.63 sentences/sec


({'BERT-P:': 1.0, 'BERT-R:': 1.0, 'BERT-F1:': 1.0},
 array([1.        , 1.0000002 , 1.        , 1.        , 0.9999997 ,
        0.99999994, 1.        , 1.        , 0.9999999 , 1.        ,
        1.        , 1.0000001 , 0.9999999 , 0.99999994, 1.        ,
        1.        , 1.        , 0.99999994, 0.9999999 , 1.        ,
        0.99999994, 0.9999999 , 1.        , 0.99999994, 0.9999999 ,
        1.0000001 , 1.        , 1.        , 0.99999994, 1.        ,
        1.0000001 , 1.        , 1.0000002 , 0.9999999 , 0.99999976,
        0.9999999 , 0.9999999 , 1.        , 1.        , 1.0000001 ,
        0.9999998 , 1.0000001 , 0.9999999 , 1.        , 1.0000001 ,
        1.0000001 , 0.9999999 , 1.        , 1.0000001 , 0.9999999 ,
        1.        , 1.        , 1.0000001 , 1.        , 1.        ,
        0.99999994, 0.9999999 , 1.        , 0.9999999 , 1.        ,
        1.        , 1.0000002 , 0.99999994, 1.        , 0.99999994,
        1.        , 0.99999994, 0.99999994, 1.        , 0.999999

# 6.Cider

In [4]:
from pycocoevalcap.cider.cider import Cider

def Cider_Score(candidates, references):

    candidates = {i: [sentence] for i, sentence in enumerate(candidates)}
    references = {i: [sentence] for i, sentence in enumerate(references)}
    cider_scorer = Cider()
    # 计算CIDEr分数
    ave_score, scores = cider_scorer.compute_score(candidates, references)
    return ave_score.item(), scores

Cider_Score(Answers, Labels)


(10.0,
 array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10.]))